In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, BatchNormalization,
    GlobalAveragePooling2D, Dense, Dropout
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from skopt import gp_minimize
from skopt.space import Integer
from skopt.utils import use_named_args
import os
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Parameters
img_size = (224, 224)
batch_size = 16  
epochs = 50  

# Data Generators
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    './train',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

valid_gen = valid_datagen.flow_from_directory(
    './valid',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# Define the search space
space = [
    Integer(32, 128, name='filters_1'),  # Filters for the first Conv2D layer
    Integer(64, 256, name='filters_2'),  # Filters for the second Conv2D layer
    Integer(128, 512, name='filters_3'),  # Filters for the third Conv2D layer
    Integer(256, 512, name='filters_4'),  # Filters for the fourth Conv2D layer
    Integer(64, 256, name='dense_units')  # Number of units in the dense layer
]

# File to save optimization progress
progress_file = 'optimized_progress.npy'

# Load previous progress 
if os.path.exists(progress_file):
    progress = np.load(progress_file, allow_pickle=True).item()
    best_params = progress['best_params']
    best_accuracy = progress['best_accuracy']
    evaluations = progress['evaluations']
    
    # Extract previous evaluations for gp_minimize
    x0 = [list(e['params'].values()) for e in evaluations]  # Hyperparameter combinations
    y0 = [-e['val_accuracy'] for e in evaluations]  # Negative validation accuracy
else:
    best_params = None
    best_accuracy = -np.inf
    evaluations = []
    x0 = None
    y0 = None

# Objective function to minimize
@use_named_args(space)
def objective(filters_1, filters_2, filters_3, filters_4, dense_units):
    """
    Build, compile, and train the model with the given hyperparameters.
    Returns the negative validation accuracy (to minimize).
    """
    global best_params, best_accuracy, evaluations

    print(f"\nTesting hyperparameters:")
    print(f"Filters: {filters_1}, {filters_2}, {filters_3}, {filters_4}")
    print(f"Dense units: {dense_units}")
    
    # Fixed hyperparameters
    learning_rate = 1e-4  # Fixed learning rate
    dropout_rate = 0.6  # Fixed dropout rate
    l2_lambda = 1e-4 # Fixed L2 regularization strength

    # Build the model
    model = Sequential([
        Conv2D(filters_1, (3, 3), activation='relu', padding='same',
               kernel_regularizer=l2(l2_lambda), input_shape=(224, 224, 3)),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        
        Conv2D(filters_2, (3, 3), activation='relu', padding='same',
               kernel_regularizer=l2(l2_lambda)),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        
        Conv2D(filters_3, (3, 3), activation='relu', padding='same',
               kernel_regularizer=l2(l2_lambda)),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        
        Conv2D(filters_4, (3, 3), activation='relu', padding='same',
               kernel_regularizer=l2(l2_lambda)),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        
        GlobalAveragePooling2D()
    ])

    # Add dense layers
    model.add(Dense(dense_units, activation='relu', kernel_regularizer=l2(l2_lambda)))
    model.add(Dropout(dropout_rate))

    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Train the model
    history = model.fit(
        train_gen,
        epochs=epochs,
        validation_data=valid_gen,
        callbacks=[
            EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        ],
        verbose=0
    )

    # Get the best validation accuracy
    val_accuracy = max(history.history['val_accuracy'])
    print(f"Validation accuracy: {val_accuracy}")

    # Update the best parameters and accuracy if this run is better
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_params = {
            'filters_1': filters_1,
            'filters_2': filters_2,
            'filters_3': filters_3,
            'filters_4': filters_4,
            'dense_units': dense_units,
            'dropout_rate': dropout_rate,
            'learning_rate': learning_rate,
            'l2_lambda': l2_lambda
        }

    # Save the current evaluation
    evaluations.append({
        'params': {
            'filters_1': filters_1,
            'filters_2': filters_2,
            'filters_3': filters_3,
            'filters_4': filters_4,
            'dense_units': dense_units
        },
        'val_accuracy': val_accuracy
    })

    # Save progress to file
    np.save(progress_file, {
        'best_params': best_params,
        'best_accuracy': best_accuracy,
        'evaluations': evaluations
    })

    # Return the negative validation accuracy (to minimize)
    return -val_accuracy

# Run Bayesian optimization
result = gp_minimize(
    func=objective,
    dimensions=space,
    n_calls=10,  # Number of evaluations
    random_state=seed,
    verbose=True,
    x0=x0,  # Previous hyperparameter combinations
    y0=y0   # Previous validation accuracies (negative)
)

# Print the best hyperparameters
print("\nBest hyperparameters:")
print(f"Filters 1: {result.x[0]}")
print(f"Filters 2: {result.x[1]}")
print(f"Filters 3: {result.x[2]}")
print(f"Filters 4: {result.x[3]}")
print(f"Dense units: {result.x[4]}")
print(f"Best validation accuracy: {-result.fun}")

# Plot validation accuracy over iterations
val_accuracies = [e['val_accuracy'] for e in evaluations]
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, marker='o')
plt.xlabel('Iteration')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy Over Iterations')
plt.show()

2025-05-07 19:06:52.935213: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746637612.990598 2964631 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746637613.007864 2964631 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-07 19:06:53.105178: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 8000 images belonging to 2 classes.
Found 770 images belonging to 2 classes.
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0006
Function value obtained: -0.8961
Current minimum: -0.9273
Iteration No: 2 started. Evaluating function at random point.

Testing hyperparameters:
Filters: 108, 99, 427, 409
Dense units: 150


/home/tiamat_hd/ThesisWork/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1746637618.068685 2964631 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5582 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


KeyboardInterrupt: 